In [7]:
# Declaration
import os
import gc
import time
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import numpy as np

from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from numpy import asarray
from numpy import ones
from numpy import zeros
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.python.keras.backend as K
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import (
    Dense,
    Input,
    LSTM,
    SimpleRNN,
    Embedding,
    Dropout,
    SpatialDropout1D,
    Activation,
    Conv1D,
    GRU,
    Reshape,
)
from keras.layers import (
    Conv1D,
    Bidirectional,
    GlobalMaxPool1D,
    MaxPooling1D,
    BatchNormalization,
    Add,
    Flatten,
)
from keras.layers import (
    GlobalMaxPooling1D,
    GlobalAveragePooling1D,
    concatenate,
    SpatialDropout1D,
)
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import CSVLogger

from tensorflow.python.distribute import distribution_strategy_context
from tensorflow.python.eager import context
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import sparse_tensor
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import check_ops
from tensorflow.python.ops import confusion_matrix
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn
from tensorflow.python.ops import sets
from tensorflow.python.ops import sparse_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.ops import variable_scope
from tensorflow.python.ops import weights_broadcast_ops
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util.deprecation import deprecated
from tensorflow.python.util.tf_export import tf_export
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import transformers
from nlpaug.util import Action

from datetime import datetime
from imblearn.over_sampling import SMOTE

import nltk


nltk.download("stopwords")
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
## Oversample Record with Data Augmentation Group by Developer and Bug Type
def CreateOversamplingWithDataAugmentation(data, splitno):
    # /*Get Contextual Word Embedding Model*/
    aug = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="substitute")
    # /*Get Developer Bug Count/
    dfcountbybug = (
        data.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"]
        .size()
        .reset_index(name="count")
    )
    dfcountbybug.to_csv("list.csv")
    # /*Get Minority Class List*/
    majoritycount = dfcountbybug[
        (dfcountbybug["FixedByID"] != "unknown") & (dfcountbybug["Name"] != "unknown")
    ]["count"].max()
    # /*Get Majority Class Count*/
    minoritylist = dfcountbybug[(dfcountbybug["count"] != dfcountbybug["count"].max())]
    print(majoritycount, len(minoritylist), majoritycount * len(minoritylist))
    estimatetotalnoofdataaugrecord = majoritycount * len(minoritylist)
    maxnoofaug = majoritycount
    ## Data Aug Record Count Validation, If over threshold reduce the majaritycount
    if estimatetotalnoofdataaugrecord > DataAugThreshold:
        print("Overtheshold--")
        maxnoofaug = int(
            (DataAugThreshold / estimatetotalnoofdataaugrecord) * majoritycount
        )

    # /*Loop through Minor Class Group*/
    for ind in minoritylist.index:
        # print(minoritylist['FixedByID'][ind], minoritylist['count'][ind])
        developer = minoritylist["FixedByID"][ind]
        bugtype = minoritylist["Name"][ind]
        minoritycount = minoritylist["count"][ind]
        data1 = data[(data["FixedByID"] == developer) & (data["Name"] == bugtype)]
        # print(len(data1), developer,bugtype)
        # print('minoritycount  --->',minoritycount, 'majoritycount--->',majoritycount, 'index --->', ind , 'out of ', len(minoritylist.index))
        # Create Sample Data until minority class count Match up with majority class count
        while minoritycount < maxnoofaug:
            # majoritycount:
            samplerow = data1.sample()
            oldbugdescription = (
                str(samplerow["Title_Description"].values).strip("[]").replace("'", "")
            )
            if oldbugdescription:
                first100words_aug = str(
                    aug.augment(" ".join(oldbugdescription.split()[:100]))
                )
                remainingwords = " ".join(oldbugdescription.split()[100:])
                newbugdescription = first100words_aug + remainingwords
                new_row = {
                    "RepoID": str(samplerow["RepoID"].values)
                    .strip("[]")
                    .replace("'", ""),
                    #'PullRequestID' : str(samplerow['PullRequestID'].values).strip('[]').replace("'",""),
                    "IssueID": str(samplerow["IssueID"].values)
                    .strip("[]")
                    .replace("'", ""),
                    "Title_Description": newbugdescription,  # /*Data Augmentation : Title_Desciption*/
                    "AST": str(samplerow["AST"].values).strip("[]").replace("'", ""),
                    "FixedByID": str(samplerow["FixedByID"].values)
                    .strip("[]")
                    .replace("'", ""),
                    "Name": str(samplerow["Name"].values).strip("[]").replace("'", ""),
                    "CreatedDate": str(samplerow["CreatedDate"].values)
                    .strip("[]")
                    .replace("'", ""),
                }
                data = data.append(new_row, ignore_index=True)
                minoritycount = minoritycount + 1
            gc.collect()
    trainfilename = DataFileName + "trainaugdata" + splitno + FileType
    data.to_csv(trainfilename)
    return data


In [9]:
def c_f1_macro(y_true, y_pred):
    """Computes 3 different f1 scores, micro macro
    weighted.
    micro: f1 score accross the classes, as 1
    macro: mean of f1 scores per class
    weighted: weighted average of f1 scores per class,
            weighted from the support of each class
    Args:
        y_true (Tensor): labels, with shape (batch, num_classes)
        y_pred (Tensor): model's predictions, same shape as y_true
    Returns:
        tuple(Tensor): (micro, macro, weighted)
                    tuple of the computed f1 scores
    """

    f1s = [0, 0, 0]

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    for i, axis in enumerate([None, 0]):
        TP = tf.math.count_nonzero(y_pred * y_true, axis=axis)
        FP = tf.math.count_nonzero(y_pred * (y_true - 1), axis=axis)
        FN = tf.math.count_nonzero((y_pred - 1) * y_true, axis=axis)

        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * precision * recall / (precision + recall)

        f1s[i] = tf.reduce_mean(f1)

    weights = tf.reduce_sum(y_true, axis=0)
    weights /= tf.reduce_sum(weights)

    f1s[2] = tf.reduce_sum(f1 * weights)

    micro, macro, weighted = f1s
    return macro


def hammingloss(y_true, y_pred, threshold=0.5, mode="multilabel"):
    """Computes hamming loss.
    Hamming loss is the fraction of wrong labels to the total number
    of labels.
    In multi-class classification, hamming loss is calculated as the
    hamming distance between `actual` and `predictions`.
    In multi-label classification, hamming loss penalizes only the
    individual labels.
    Args:
        y_true: actual target value
        y_pred: predicted target value
        threshold: Elements of `y_pred` greater than threshold are
            converted to be 1, and the rest 0. If threshold is
            None, the argmax is converted to 1, and the rest 0.
        mode: multi-class or multi-label
    Returns:
        hamming loss: float
    Usage:
    ```python
    # multi-class hamming loss
    hl = HammingLoss(mode='multiclass', threshold=0.6)
    actuals = tf.constant([[1, 0, 0, 0],[0, 0, 1, 0],
                       [0, 0, 0, 1],[0, 1, 0, 0]],
                      dtype=tf.float32)
    predictions = tf.constant([[0.8, 0.1, 0.1, 0],
                               [0.2, 0, 0.8, 0],
                               [0.05, 0.05, 0.1, 0.8],
                               [1, 0, 0, 0]],
                          dtype=tf.float32)
    hl.update_state(actuals, predictions)
    print('Hamming loss: ', hl.result().numpy()) # 0.25
    # multi-label hamming loss
    hl = HammingLoss(mode='multilabel', threshold=0.8)
    actuals = tf.constant([[1, 0, 1, 0],[0, 1, 0, 1],
                       [0, 0, 0,1]], dtype=tf.int32)
    predictions = tf.constant([[0.82, 0.5, 0.90, 0],
                               [0, 1, 0.4, 0.98],
                               [0.89, 0.79, 0, 0.3]],
                               dtype=tf.float32)
    hl.update_state(actuals, predictions)
    print('Hamming loss: ', hl.result().numpy()) # 0.16666667
    ```
    """
    if mode not in ["multiclass", "multilabel"]:
        raise TypeError("mode must be either multiclass or multilabel]")

    if threshold is None:
        threshold = tf.reduce_max(y_pred, axis=-1, keepdims=True)
        # make sure [0, 0, 0] doesn't become [1, 1, 1]
        # Use abs(x) > eps, instead of x != 0 to check for zero
        y_pred = tf.logical_and(y_pred >= threshold, tf.abs(y_pred) > 1e-12)
    else:
        y_pred = y_pred > threshold

    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)

    if mode == "multiclass":
        nonzero = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=-1), tf.float32)
        return 1.0 - nonzero

    else:
        nonzero = tf.cast(tf.math.count_nonzero(y_true - y_pred, axis=-1), tf.float32)
        return nonzero / y_true.get_shape()[-1]


def c_precision(y_true, y_pred):
    """Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    true_positives = tf.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), tf.float32)
    predicted_positives = tf.cast(K.sum(K.round(K.clip(y_pred, 0, 1))), tf.float32)
    precision = tf.cast(
        true_positives / (predicted_positives + K.epsilon()), tf.float32
    )
    return precision


def c_recall(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    """Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = tf.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), tf.float32)
    possible_positives = tf.cast(K.sum(K.round(K.clip(y_true, 0, 1))), tf.float32)
    recall = tf.cast(true_positives / (possible_positives + K.epsilon()), tf.float32)
    return recall


def c_fbeta(y_true, y_pred, beta=2):
    y_pred = backend.clip(y_pred, 0, 1)
    # calculate elements
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
    # calculate precision
    p = tp / (tp + fp + backend.epsilon())
    # calculate recall
    r = tp / (tp + fn + backend.epsilon())
    # calculate fbeta, averaged across each class
    bb = beta**2
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score


def c_auroc(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    auc = tf.cast(tf.metrics.auc(y_true, y_pred)[1], tf.float32)
    K.get_session().run(tf.local_variables_initializer())
    return auc


In [10]:
# Remove stopwords
def Cleandata(data):
    porter = PorterStemmer()
    for ind in data.index:
        bug_description = data["Title_Description"][ind]
        bug_description = re.sub("[^a-zA-Z]", " ", str(bug_description))
        if bug_description and bug_description.strip():
            tk_bug_description = word_tokenize(bug_description)
            tokens_without_sw = (
                str(
                    [
                        word
                        for word in tk_bug_description
                        if not word in stopwords.words("english")
                    ]
                )
                .strip("[]")
                .replace("'", "")
                .replace(",", "")
                .replace("\\", "")
            )
            tokens_without_sl = " ".join(
                [w for w in tokens_without_sw.split() if len(w) > 1]
            )  # token without single letter
            tokens_with_stemm = " ".join(
                [porter.stem(w) for w in tokens_without_sl.split()]
            )  # token with stemm
            # data['Title_Description'][ind] = tokens_with_stemm
            data.loc[ind, "Title_Description"] = tokens_with_stemm
    return data


def create_tag_mapping(mapping_csv, tagname):
    print("tagname", tagname)
    # create a set of all known tags
    labels = set()
    IssueType_Tags = []
    for i in range(len(mapping_csv)):
        # convert spaced separated tags into an array of tags
        tags = mapping_csv[i].split("| ")
        # add tags to the set of known labels
        labels.update(tags)

    # convert set of labels to a list to list
    labels = list(labels)
    # order set alphabetically
    labels.sort()
    # dict that maps labels to integers, and the reverse
    labels_map = {labels[i]: i for i in range(len(labels))}
    inv_labels_map = {i: labels[i] for i in range(len(labels))}

    for i in range(len(mapping_csv)):
        # Create One Hot Encoding For Issue Type
        IssueType_Tag = one_hot_encode(mapping_csv[i].split("| "), labels_map)
        IssueType_Tags.append(IssueType_Tag)

    result = IssueType_Tags
    return labels_map, inv_labels_map, result


#########SplitTrainTestData################
def SplitTrainTestData(data):
    x_train_context = []
    x_train_AST = []
    dev_y_train = []
    btype_y_train = []
    x_test_context = []
    x_test_AST = []
    dev_y_test = []
    btype_y_test = []
    tk_context = []

    dev_y = list(data["FixedByID"].astype(str))  # Developer List
    btype_y = list(data["Name"].astype(str))  # Bug Type List
    data.Title_Description = data.Title_Description.astype(str)
    x_context = list(data["Title_Description"])
    data.AST = data.AST.astype(str)
    x_AST = list(data["AST"])

    # 80% / 20% train / test split:
    train_size = int(len(x_context) * 0.8)

    x_train_context = x_context[:train_size]
    x_train_AST = x_AST[:train_size]
    dev_y_train = dev_y[:train_size]
    btype_y_train = btype_y[:train_size]

    x_test_context = x_context[train_size:]
    x_test_AST = x_AST[train_size:]
    dev_y_test = dev_y[train_size:]
    btype_y_test = btype_y[train_size:]

    # /*******************Label Encoder***********************/
    ### Developer Encoder
    combineddata_dev = dev_y_train + dev_y_test
    dev_labels_map, dev_inv_labels_map, combineddata_dev_enc = create_tag_mapping(
        combineddata_dev, "FixedByID"
    )
    dev_y_train = combineddata_dev_enc[: len(dev_y_train)]
    dev_y_test = combineddata_dev_enc[len(dev_y_train) :]
    print(
        "Developer",
        "Training: ",
        len(dev_y_train),
        "Testing :",
        len(dev_y_test),
        "Combined DEV + TEST",
        len(combineddata_dev_enc),
    )

    ### BugType Encoder
    combineddata_bugtype = btype_y_train + btype_y_test
    (
        btype_labels_map,
        btype_inv_labels_map,
        combineddata_bugtype_enc,
    ) = create_tag_mapping(combineddata_bugtype, "Name")
    btype_y_train = combineddata_bugtype_enc[: len(btype_y_train)]
    btype_y_test = combineddata_bugtype_enc[len(btype_y_train) :]
    print(
        "Bug Type",
        "Training: ",
        len(btype_y_train),
        "Testing :",
        len(btype_y_test),
        "Combined DEV + TEST",
        len(combineddata_bugtype_enc),
    )

    # Tokenizer
    x_train_context = [str(row).lower() for row in x_train_context]
    x_test_context = [str(row).lower() for row in x_test_context]
    combineddata = x_train_context + x_test_context
    tk_context = Tokenizer(
        num_words=None,
        char_level=None,
        oov_token="Unknown",
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\d+',
    )
    tk_context.fit_on_texts(combineddata)

    return (
        tk_context,
        x_train_context,
        x_train_AST,
        dev_y_train,
        btype_y_train,
        x_test_context,
        x_test_AST,
        dev_y_test,
        btype_y_test,
        dev_labels_map,
        dev_inv_labels_map,
        btype_labels_map,
        btype_inv_labels_map,
    )


def SplitTrainTestDataForMultiLabelWithDataAugmentation(data, splitno):
    x_train_context = []
    x_train_AST = []
    dev_y_train = []
    btype_y_train = []
    x_test_context = []
    x_test_AST = []
    dev_y_test = []
    btype_y_test = []
    tk_context = []

    if LoadDataAugFromFile == True:
        traindata = pd.read_csv(
            DataFilePath
            + "DataAugmentation/"
            + DataFileName
            + "trainaugdata"
            + splitno
            + FileType,
            error_bad_lines=False,
            index_col=False,
            dtype="unicode",
            encoding="latin-1",
            low_memory=False,
        ).sample(frac=1)
        traindata = traindata.rename(columns={"ï»¿RepoID": "RepoID"}, inplace=False)

        testdata = pd.read_csv(
            DataFilePath
            + "DataAugmentation/"
            + DataFileName
            + "testdata"
            + splitno
            + FileType,
            error_bad_lines=False,
            index_col=False,
            dtype="unicode",
            encoding="latin-1",
            low_memory=False,
        ).sample(frac=1)
        testdata = testdata.rename(columns={"ï»¿RepoID": "RepoID"}, inplace=False)
        testdata = RemoveTestRecordIfNotExistInTrainData(traindata, testdata)
    else:
        # 80% / 20% train / test split:
        train_size = int(len(data) * 0.8)
        traindata = data[:train_size]
        testdata = data[train_size:]
        # Save Data to TestFile
        testfilename = DataFileName + "testdata" + splitno + FileType
        testdata.to_csv(testfilename)
        # /*Create Sample Data*/
        traindata = CreateOversamplingWithDataAugmentation(traindata, splitno)

    # /*****************Train Data***********************/
    train_dev_y = list(traindata["FixedByID"])  # Developer List
    train_btype_y = list(traindata["Name"])  # Bug Type List
    train_x_context = list(traindata["Title_Description"])
    traindata.AST = traindata.AST.astype(str)
    train_x_AST = list(traindata["AST"])

    x_train_context = train_x_context
    x_train_AST = train_x_AST

    # /****************Test Data************************/
    x_test_context = list(testdata["Title_Description"])
    x_test_AST = list(testdata["Title_Description"])
    test_dev_y = list(testdata["FixedByID"])  # Developer List
    test_btype_y = list(testdata["Name"])  # Bug Type List

    # /*******************Label Encoder***********************/
    ### Developer Encoder
    combineddata_dev = train_dev_y + test_dev_y
    dev_labels_map, dev_inv_labels_map, combineddata_dev_enc = create_tag_mapping(
        combineddata_dev, "FixedByID"
    )
    dev_y_train = combineddata_dev_enc[: len(train_dev_y)]
    dev_y_test = combineddata_dev_enc[len(train_dev_y) :]
    print(
        "Developer",
        "Training: ",
        len(train_dev_y),
        "Testing :",
        len(test_dev_y),
        "Combined DEV + TEST",
        len(combineddata_dev_enc),
    )

    ### BugType Encoder
    combineddata_bugtype = train_btype_y + test_btype_y
    (
        btype_labels_map,
        btype_inv_labels_map,
        combineddata_bugtype_enc,
    ) = create_tag_mapping(combineddata_bugtype, "Name")
    btype_y_train = combineddata_bugtype_enc[: len(train_btype_y)]
    btype_y_test = combineddata_bugtype_enc[len(train_btype_y) :]
    print(
        "Bug Type",
        "Training: ",
        len(btype_y_train),
        "Testing :",
        len(btype_y_test),
        "Combined DEV + TEST",
        len(combineddata_bugtype_enc),
    )

    # /*******************Tokenizer****************************/

    x_train_context = [str(row).lower() for row in x_train_context]
    x_test_context = [str(row).lower() for row in x_test_context]
    combineddata = x_train_context + x_test_context
    tk_context = Tokenizer(
        num_words=None,
        char_level=None,
        oov_token="Unknown",
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\d+',
    )
    tk_context.fit_on_texts(combineddata)

    return (
        traindata,
        tk_context,
        x_train_context,
        x_train_AST,
        dev_y_train,
        btype_y_train,
        x_test_context,
        x_test_AST,
        dev_y_test,
        btype_y_test,
        dev_labels_map,
        dev_inv_labels_map,
        btype_labels_map,
        btype_inv_labels_map,
    )


def Tokenize(tk_context, x_train_context, x_train_AST, x_test_context, x_test_AST):
    # =======================Convert string to index================
    # Tokenizer
    x_train_context = [str(row).lower() for row in x_train_context]
    x_test_context = [str(row).lower() for row in x_test_context]
    combineddata_context = x_train_context + x_test_context
    tk_context = Tokenizer(
        num_words=None,
        char_level=None,
        oov_token="Unknown",
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    )
    tk_context.fit_on_texts(combineddata_context)

    x_train_AST = [str(row).lower() for row in x_train_AST]
    x_test_AST = [str(row).lower() for row in x_test_AST]
    combineddata_AST = x_train_AST + x_test_AST
    tk_AST = Tokenizer(num_words=None, char_level=None, oov_token="Unknown")
    tk_AST.fit_on_texts(combineddata_AST)

    # Convert string to index
    x_train_context_sequences = tk_context.texts_to_sequences(x_train_context)
    x_train_AST_sequences = tk_AST.texts_to_sequences(x_train_AST)
    x_test_context_sequences = tk_context.texts_to_sequences(x_test_context)
    x_test_AST_sequences = tk_AST.texts_to_sequences(x_test_AST)

    # Padding
    x_train_context = pad_sequences(
        x_train_context_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post"
    )
    x_train_AST = pad_sequences(
        x_train_AST_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post"
    )
    x_test_context = pad_sequences(
        x_test_context_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post"
    )
    x_test_AST = pad_sequences(
        x_test_AST_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post"
    )

    # Convert to numpy array
    x_train_context = np.array(x_train_context)
    x_train_AST = np.array(x_train_AST)
    x_test_context = np.array(x_test_context)
    x_test_AST = np.array(x_test_AST)
    return tk_context, tk_AST, x_train_context, x_train_AST, x_test_context, x_test_AST


## Remove record if does not exist in TrainData
def RemoveTestRecordIfNotExistInTrainData(traindata, testdata):
    traingroup = (
        traindata.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"]
        .size()
        .reset_index(name="count")
    )
    testgroup = (
        testdata.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"]
        .size()
        .reset_index(name="count")
    )
    for ind in testgroup.index:
        try:
            record = traindata[
                traindata["FixedByID"].str.match(testgroup["FixedByID"][ind])
                & traindata["Name"].str.match(testgroup["Name"][ind])
            ]
            if len(record) < 1:
                print("remove from testdata...")
                testdata = testdata.drop(
                    testdata[
                        testdata["FixedByID"].str.match(testgroup["FixedByID"][ind])
                        & testdata["Name"].str.match(testgroup["Name"][ind])
                    ].index
                )
        except:
            print("An exception occurred index :", ind)
    return testdata


# create a one hot encoding for one list of tags
def one_hot_encode(tags, mapping):
    # create empty vector
    encoding = zeros(len(mapping), dtype="uint8")
    # mark 1 for each tag in the vector
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding


In [11]:
def MultiModel(
    project,
    btype_labels_map,
    dev_labels_map,
    btype_y_train,
    dev_y_train,
    btype_y_test,
    dev_y_test,
    dataaug,
    MAX_SEQUENCE_LENGTH,
    EMBEDDING_DIM,
):
    # Logging
    filename = (
        "Multimodel"
        + "_"
        + project
        + "_"
        + dataaug
        + "_"
        + str(Learningrate)
        + "_"
        + str(EMBEDDING_DIM)
        + "_"
        + str(MAX_SEQUENCE_LENGTH)
    )
    filelog = open(filename + ".txt", "w")
    filelog.write("StartTime:" + str(datetime.now()))
    filelog.close()

    # Paramaters
    VALIDATION_SPLIT = 0.2
    noofbugtype = len(btype_labels_map)
    noofdev = len(dev_labels_map)
    btype_y_train = np.array(btype_y_train)
    dev_y_train = np.array(dev_y_train)
    btype_y_test = np.array(btype_y_test)
    dev_y_test = np.array(dev_y_test)

    # Visualize Model
    logdir = "logs/"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    # A model.fit() training loop will check at end of every epoch whether the loss is no longer decreasing, considering the min_delta and patiencez
    earlystop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)

    starttime = datetime.now()
    print("Start Time =", starttime)
    print("Predict Developer")
    with tf.device('/GPU:0'):
        # inputs
        input_context = Input(
            shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name="Bug_TitleandDescription"
        )  # Bug Title and Description
        input_AST = Input(
            shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name="Bug_CodeSnippetAST"
        )  # Bug Code Snippet AST

        # Context Enconder
        emb_Context = Embedding(
            input_dim=len(tk_context.word_index) + 2,
            input_length=MAX_SEQUENCE_LENGTH,
            output_dim=EMBEDDING_DIM,
            name="Context_Embedding",
        )(input_context)
        conv_Context = Conv1D(
            filters=64,
            kernel_size=2,
            padding="same",
            activation="relu",
            name="Context_Convolutional_Layer",
        )(emb_Context)
        maxpool_Context = GlobalMaxPool1D(name="Context_Maxpool_Layer")(conv_Context)
        flatcon = Flatten(name="Context_Flatten_Layer")(maxpool_Context)

        # AST Enconder
        emb_AST = Embedding(
            input_dim=len(tk_AST.word_index) + 2,
            input_length=MAX_SEQUENCE_LENGTH,
            output_dim=EMBEDDING_DIM,
            name="AST_Embedding",
        )(input_AST)
        bilstm_AST = Bidirectional(LSTM(25, return_sequences=True, name="AST_LSTM_Layer"))(
            emb_AST
        )
        maxpool_AST = GlobalMaxPool1D(name="AST_Maxpool_Layer")(bilstm_AST)
        flatAST = Flatten(name="AST_Flatten_Layer")(maxpool_AST)

        cat = concatenate([flatcon, flatAST], name="Concatenate_Flatten_Layer")

        bn = BatchNormalization()(cat)
        drop = Dropout(0.5)(bn)
        dense = Dense(50, activation="relu")(drop)
        DevOutput = Dense(noofdev, activation="sigmoid", name="Developer")(dense)
        BugTypeOutput = Dense(noofbugtype, activation="sigmoid", name="Bug_Type")(dense)
        Bil_LSTM_MultiTask_model = Model(
            inputs=[input_context, input_AST], outputs=[DevOutput, BugTypeOutput]
        )

        Bil_LSTM_MultiTask_model.compile(
            loss="binary_crossentropy",
            optimizer=Adam(Learningrate),
            metrics=[
                "accuracy",
                c_precision,
                c_recall,
                "AUC",
                c_f1_macro,
                c_fbeta,
                hammingloss,
            ],
        )

        Bil_LSTM_MultiTask_model.fit(
            [x_train_context, x_train_AST],
            [dev_y_train, btype_y_train],
            callbacks=[tensorboard_callback, earlystop],
            epochs=50,
            verbose=2,
            validation_split=0.1,
        )

    ## Evaluate RNN Single Model
        csv_logger = CSVLogger(filename + ".csv", append=True, separator=";")
        history = Bil_LSTM_MultiTask_model.evaluate(
            [x_test_context, x_test_AST], [dev_y_test, btype_y_test], callbacks=[csv_logger]
        )
    return history
    # filelog =  open(filename + ".txt","a")
    # filelog.write("Endtime:" + str(datetime.now()))
    # filelog.close()


In [12]:
# Setup Project Parameters
# DataAugmentation = True
DataAugmentation = False

DataAugThreshold = 30000
DataFilePath = "./Data/"
DataFileName = "IssueelasticsearchWebScrap"  # Replace with Project File Name
FileType = ".csv"
MAX_SEQUENCE_LENGTH = 300
EMBEDDING_DIM = 100
LoadDataAugFromFile = False
Learningrate = 0.001

for DataFileName in os.listdir('./Data'):
    if not 'aspnetcore' in DataFileName:
        continue
    DataFileName = DataFileName.replace('.csv', '')
# Loading Data

    totaldata = pd.read_csv(
        DataFilePath + DataFileName + FileType,
        
        index_col=False,
        dtype="unicode",
        encoding="latin-1",
        low_memory=False,
    ).sample(frac=1)
    totaldata = totaldata.rename(columns={"ï»¿RepoID": "RepoID"}, inplace=False)
    totaldata["CreatedDate"] = pd.to_datetime(totaldata["CreatedDate"])
    totaldata = totaldata.sort_values(by=["CreatedDate"])
    # totaldata[totaldata['FixedByID'] != 'unknown']


    # Cross Validation in TimeSeries
    totalnoofrecords = len(totaldata)
    split = int(totalnoofrecords / 5)
    print(split)
    mm_res = pd.DataFrame([])
    res_metrics = [
        "loss",
        "Developer_loss",
        "Bug_Type_loss",
        "Developer_accuracy",
        "Developer_c_precision",
        "Developer_c_recall",
        "Developer_auc",
        "Developer_c_f1_macronan",
        "Developer_c_fbeta",
        "Developer_hammingloss",
        "Bug_Type_accuracy",
        "Bug_Type_c_precision",
        "Bug_Type_c_recall",
        "Bug_Type_auc_1",
        "Bug_Type_c_f1_macronan",
        "Bug_Type_c_fbeta",
        "Bug_Type_hammingloss",
    ]
    for i in [1, 2, 3, 4, 5]:
        # for i in [5]:
        currentsplit = split * i
        data = totaldata[: int(currentsplit)]
        Cleandata(data)
        if DataAugmentation == True:
            (
                traindata,
                tk_context,
                x_train_context,
                x_train_AST,
                dev_y_train,
                btype_y_train,
                x_test_context,
                x_test_AST,
                dev_y_test,
                btype_y_test,
                dev_labels_map,
                dev_inv_labels_map,
                btype_labels_map,
                btype_inv_labels_map,
            ) = SplitTrainTestDataForMultiLabelWithDataAugmentation(data, str(i))
        else:
            (
                tk_context,
                x_train_context,
                x_train_AST,
                dev_y_train,
                btype_y_train,
                x_test_context,
                x_test_AST,
                dev_y_test,
                btype_y_test,
                dev_labels_map,
                dev_inv_labels_map,
                btype_labels_map,
                btype_inv_labels_map,
            ) = SplitTrainTestData(data)
        (
            tk_context,
            tk_AST,
            x_train_context,
            x_train_AST,
            x_test_context,
            x_test_AST,
        ) = Tokenize(tk_context, x_train_context, x_train_AST, x_test_context, x_test_AST)
        history = MultiModel(
            DataFileName + "_30_epoch",
            btype_labels_map,
            dev_labels_map,
            btype_y_train,
            dev_y_train,
            btype_y_test,
            dev_y_test,
            str(DataAugmentation),
            MAX_SEQUENCE_LENGTH,
            EMBEDDING_DIM,
        )
        mm_res = pd.concat(
            [mm_res, pd.Series(dict(zip(res_metrics, history)))], axis=1, ignore_index=True
        )
        print("finished iteration", i)
    mm_res.T.to_csv("./logs/res/" + DataFileName + ".csv")


ValueError: time data "19/01/2016" doesn't match format "%m/%d/%Y", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
class PredictCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        evaluate = self.model.evaluate([x_test_context, x_test_AST], np.array(dev_y_test))
        df_evaluate = pd.DataFrame(columns=['loss','accuracy','c_precision','c_recall', 'auc', 'c_fbeta', 'c_f1_macro', 'hammingloss'])
        df_row = {'loss': evaluate[0], 'accuracy': evaluate[1],'c_precision': evaluate[2],
                  'c_recall': evaluate[3], 'auc': evaluate[4], 'c_fbeta': evaluate[5],
                  'c_f1_macro': evaluate[6], 'hammingloss': evaluate[7]}
        print(df_row)



def CNNModel(btype_labels_map,dev_labels_map,btype_y_train,dev_y_train,btype_y_test,dev_y_test,splitno,DEV_PREDICT):
  from datetime import datetime
  #Logging 
  LR = str(Learningrate).replace('.','')

  if DEV_PREDICT == True: 
    predictwhat = 'DEVPREDICT'
  else:
    predictwhat = 'BUGPREDICT'

  if DataAugmentation == True:
   filename = DataFilePath + 'Results/CNNModel' + '_' + DataFileName + '_' + predictwhat + '_' + 'DataAug' + '_' + LR + '_' + str(EMBEDDING_DIM) + '_' + str(MAX_SEQUENCE_LENGTH) + '_S' +  str(splitno)
  else:
   filename = DataFilePath +'Results/CNNModel' + '_' + DataFileName + '_' + predictwhat + '_' + LR + '_' + str(EMBEDDING_DIM) + '_' + str(MAX_SEQUENCE_LENGTH) + '_S' + str(splitno)

  filelog = open(filename + ".txt", "w")
  filelog.write("StartTime:" + str(datetime.now()))
  filelog.close()
  
  VALIDATION_SPLIT = 0.2
  noofbugtype = len(btype_labels_map)
  noofdev = len(dev_labels_map)
  btype_y_train = np.array(btype_y_train)
  dev_y_train = np.array(dev_y_train)
  btype_y_test = np.array(btype_y_test)
  dev_y_test = np.array(dev_y_test)

  #Visualize Model
  logdir = "logs/"
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
  #A model.fit() training loop will check at end of every epoch whether the loss is no longer decreasing, considering the min_delta and patiencez
  earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

  if DEV_PREDICT == True:
    print('Predict Developer')
      # inputs
    input_context = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name="Bug_TitleandDescription") #Bug Title and Description 
    input_AST = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name ="Bug_CodeSnippetAST") #Bug Code Snippet AST
    emb_Context = Embedding(input_dim=len(tk_context.word_index) + 2, input_length=MAX_SEQUENCE_LENGTH, output_dim= EMBEDDING_DIM)(input_context)
    emb_AST = Embedding(input_dim=len(tk_AST.word_index) + 2, input_length=MAX_SEQUENCE_LENGTH, output_dim= EMBEDDING_DIM)(input_AST)
    cat = concatenate([emb_Context, emb_AST])
    sd = SpatialDropout1D(0.5)(cat)
    con = Conv1D(filters=100, kernel_size=4, padding='same', activation='relu')(sd)
    bn = BatchNormalization()(con)
    drop = Dropout(0.5)(bn)
    maxpool = GlobalMaxPool1D()(drop)
    dense = Dense(50,activation = 'relu')(maxpool)
    final = Dense(noofdev, activation = 'sigmoid') (dense)
    CNN_model = Model(inputs=[input_context, input_AST], outputs=[final])
    ## Evaluate CNN Single Model 
    CNN_model.compile(loss='binary_crossentropy', optimizer=Adam(Learningrate), metrics=['accuracy',c_precision,c_recall,'AUC',c_fbeta, c_f1_macro, hammingloss])
    csv_logger = CSVLogger(filename + '.csv', append=True, separator=',')
    history = CNN_model.fit([x_train_context, x_train_AST], 
              dev_y_train, 
              callbacks=[tensorboard_callback,earlystop,csv_logger,PredictCallback()],
              epochs=15, verbose=2, validation_split= 0.1)
    
    ## Evaluate CNN Single Model 
    evaluate = CNN_model.evaluate([x_test_context, x_test_AST], dev_y_test)
    df_evaluate = pd.DataFrame(columns=['loss','accuracy','c_precision','c_recall', 'auc', 'c_fbeta', 'c_f1_macro', 'hammingloss'])
    df_row = {'loss': evaluate[0], 'accuracy': evaluate[1],'c_precision': evaluate[2],
              'c_recall': evaluate[3], 'auc': evaluate[4], 'c_fbeta': evaluate[5],
              'c_f1_macro': evaluate[6], 'hammingloss': evaluate[7]}
    df_evaluate= df_evaluate.append(df_row, ignore_index=True)
    df_evaluate.to_csv(filename + "_eval.csv")

    filelog =  open(filename + ".txt","a")
    filelog.write("Endtime:" + str(datetime.now()))
    filelog.close() 

  else: 
    print('Predict Bug Type')
    # inputs
    input_context = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name="Bug_TitleandDescription") #Bug Title and Description 
    input_AST = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name ="Bug_CodeSnippetAST") #Bug Code Snippet AST
    emb_Context = Embedding(input_dim=len(tk_context.word_index) + 2, input_length=MAX_SEQUENCE_LENGTH, output_dim= EMBEDDING_DIM)(input_context)
    emb_AST = Embedding(input_dim=len(tk_AST.word_index) + 2, input_length=MAX_SEQUENCE_LENGTH, output_dim= EMBEDDING_DIM)(input_AST)
    cat = concatenate([emb_Context, emb_AST])
    sd = SpatialDropout1D(0.5)(cat)
    con = Conv1D(filters=100, kernel_size=4, padding='same', activation='relu')(sd)
    bn = BatchNormalization()(con)
    drop = Dropout(0.5)(bn)
    maxpool = GlobalMaxPool1D()(drop)
    dense = Dense(50,activation = 'relu')(maxpool)
    final = Dense(noofbugtype, activation = 'sigmoid') (dense)
    CNN_model = Model(inputs=[input_context, input_AST], outputs=[final])
    ## Evaluate CNN Single Model 
    CNN_model.compile(loss='binary_crossentropy', optimizer=Adam(Learningrate), metrics=['accuracy',c_precision,c_recall,'AUC',c_fbeta, c_f1_macro, hammingloss])
    csv_logger = CSVLogger(filename + '.csv', append=True, separator=',')
    history = CNN_model.fit([x_train_context, x_train_AST], 
              btype_y_train, 
              callbacks=[tensorboard_callback,earlystop,csv_logger],
              epochs=50, verbose=2, validation_split= 0.1)
    
    ## Evaluate RNN Single Model 
    # evaluate = CNN_model.evaluate([x_test_context, x_test_AST], btype_y_test)
    evaluate = CNN_model.evaluate([x_train_context, x_train_AST], btype_y_train)
    df_evaluate = pd.DataFrame(columns=['loss','accuracy','c_precision','c_recall', 'auc', 'c_fbeta', 'c_f1_macro', 'hammingloss'])
    df_row = {'loss': evaluate[0], 'accuracy': evaluate[1],'c_precision': evaluate[2],
              'c_recall': evaluate[3], 'auc': evaluate[4], 'c_fbeta': evaluate[5],
              'c_f1_macro': evaluate[6], 'hammingloss': evaluate[7]}
    df_evaluate= df_evaluate.append(df_row, ignore_index=True)
    df_evaluate.to_csv(filename + "_eval.csv")

    filelog =  open(filename + ".txt","a")
    filelog.write("Endtime:" + str(datetime.now()))
    filelog.close()

In [ ]:
CNNModel(btype_labels_map,dev_labels_map,btype_y_train,dev_y_train,btype_y_test,dev_y_test,5,False)

Predict Bug Type
Epoch 1/50
27/27 - 1s - loss: 0.3686 - accuracy: 0.3090 - c_precision: 0.2702 - c_recall: 0.3006 - auc: 0.7032 - c_fbeta: 0.2824 - c_f1_macro: nan - hammingloss: 0.1240 - val_loss: 0.6207 - val_accuracy: 0.0430 - val_c_precision: 0.0404 - val_c_recall: 0.0318 - val_auc: 0.6519 - val_c_fbeta: 0.0254 - val_c_f1_macro: nan - val_hammingloss: 0.1246 - 1s/epoch - 53ms/step
Epoch 2/50
27/27 - 0s - loss: 0.1477 - accuracy: 0.3737 - c_precision: 0.3992 - c_recall: 0.1915 - auc: 0.8612 - c_fbeta: 0.1968 - c_f1_macro: nan - hammingloss: 0.0496 - val_loss: 0.5949 - val_accuracy: 0.0538 - val_c_precision: 0.0873 - val_c_recall: 0.0263 - val_auc: 0.7605 - val_c_fbeta: 0.0227 - val_c_f1_macro: nan - val_hammingloss: 0.0874 - 213ms/epoch - 8ms/step
Epoch 3/50
27/27 - 0s - loss: 0.1293 - accuracy: 0.4323 - c_precision: 0.5203 - c_recall: 0.2504 - auc: 0.8942 - c_fbeta: 0.2639 - c_f1_macro: nan - hammingloss: 0.0463 - val_loss: 0.5808 - val_accuracy: 0.1183 - val_c_precision: 0.0641 - 

/tmp/ipykernel_2176299/717167366.py:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_evaluate= df_evaluate.append(df_row, ignore_index=True)


In [ ]:
np.array(dev_y_test).argmax(1)
unique, counts = np.unique(np.array(dev_y_test).argmax(1), return_counts=True)
sorted(dict(zip(unique, counts)).items(), key=lambda x:x[1])

[(9, 1), (4, 3), (12, 3), (7, 4), (11, 6), (5, 21), (14, 194)]

In [ ]:
# Single Task RNN Model
def RNNModel(
    btype_labels_map,
    dev_labels_map,
    btype_y_train,
    dev_y_train,
    btype_y_test,
    dev_y_test,
    splitno,
    DEV_PREDICT,
):
    from datetime import datetime

    # Logging
    LR = str(Learningrate).replace(".", "")

    if DEV_PREDICT == True:
        predictwhat = "DEVPREDICT"
    else:
        predictwhat = "BUGPREDICT"

    if DataAugmentation == True:
        filename = (
            DataFilePath
            + "Results/RNNModel"
            + "_"
            + DataFileName
            + "_"
            + predictwhat
            + "_"
            + "DataAug"
            + "_"
            + LR
            + "_"
            + str(EMBEDDING_DIM)
            + "_"
            + str(MAX_SEQUENCE_LENGTH)
            + "_S"
            + str(splitno)
        )
    else:
        filename = (
            DataFilePath
            + "Results/RNNModel"
            + "_"
            + DataFileName
            + "_"
            + predictwhat
            + "_"
            + LR
            + "_"
            + str(EMBEDDING_DIM)
            + "_"
            + str(MAX_SEQUENCE_LENGTH)
            + "_S"
            + str(splitno)
        )

    filelog = open(filename + ".txt", "w")
    filelog.write("StartTime:" + str(datetime.now()))
    filelog.close()

    VALIDATION_SPLIT = 0.2
    noofbugtype = len(btype_labels_map)
    noofdev = len(dev_labels_map)
    btype_y_train = np.array(btype_y_train)
    dev_y_train = np.array(dev_y_train)
    btype_y_test = np.array(btype_y_test)
    dev_y_test = np.array(dev_y_test)

    # Visualize Model
    logdir = "logs/"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    # A model.fit() training loop will check at end of every epoch whether the loss is no longer decreasing, considering the min_delta and patiencez
    earlystop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)

    if DEV_PREDICT == True:
        print("Predict Developer")
        # inputs
        input_context = Input(
            shape=(MAX_SEQUENCE_LENGTH,),
            dtype=tf.float32,
            name="Bug_TitleandDescription",
        )  # Bug Title and Description
        input_AST = Input(
            shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name="Bug_CodeSnippetAST"
        )  # Bug Code Snippet AST
        emb_Context = Embedding(
            input_dim=len(tk_context.word_index) + 2,
            input_length=MAX_SEQUENCE_LENGTH,
            output_dim=EMBEDDING_DIM,
        )(input_context)
        emb_AST = Embedding(
            input_dim=len(tk_AST.word_index) + 2,
            input_length=MAX_SEQUENCE_LENGTH,
            output_dim=EMBEDDING_DIM,
        )(input_AST)
        cat = concatenate([emb_Context, emb_AST])
        sd = SpatialDropout1D(0.5)(cat)
        # Bidirectional layer will enable our model to predict a missing word in a sequence,
        # So, using this feature will enable the model to look at the context on both the left and the right.
        bilstm = Bidirectional(LSTM(25, return_sequences=True))(sd)
        bn = BatchNormalization()(bilstm)
        drop = Dropout(0.5)(bn)
        maxpool = GlobalMaxPool1D()(drop)
        dense = Dense(50, activation="relu")(maxpool)
        final = Dense(noofdev, activation="sigmoid")(dense)
        Bil_LSTM_model = Model(inputs=[input_context, input_AST], outputs=[final])

        Bil_LSTM_model.compile(
            loss="binary_crossentropy",
            optimizer=Adam(Learningrate),
            metrics=[
                "accuracy",
                c_precision,
                c_recall,
                "AUC",
                c_fbeta,
                c_f1_macro,
                hammingloss,
            ],
        )

        csv_logger = CSVLogger(filename + ".csv", append=True, separator=",")
        history = Bil_LSTM_model.fit(
            [x_train_context, x_train_AST],
            dev_y_train,
            callbacks=[tensorboard_callback, earlystop, csv_logger],
            epochs=50,
            verbose=2,
            validation_split=0.1,
        )

        ## Evaluate RNN Single Model

        evaluate = Bil_LSTM_model.evaluate([x_test_context, x_test_AST], dev_y_test)
        df_evaluate = pd.DataFrame(
            columns=[
                "loss",
                "accuracy",
                "c_precision",
                "c_recall",
                "auc",
                "c_fbeta",
                "c_f1_macro",
                "hammingloss",
            ]
        )
        df_row = {
            "loss": evaluate[0],
            "accuracy": evaluate[1],
            "c_precision": evaluate[2],
            "c_recall": evaluate[3],
            "auc": evaluate[4],
            "c_fbeta": evaluate[5],
            "c_f1_macro": evaluate[6],
            "hammingloss": evaluate[7],
        }
        df_evaluate = df_evaluate.append(df_row, ignore_index=True)
        df_evaluate.to_csv(filename + "_eval.csv")

        filelog = open(filename + ".txt", "a")
        filelog.write("Endtime:" + str(datetime.now()))
        filelog.close()

    else:
        print("Predict Bug Type")
        # inputs
        input_context = Input(
            shape=(MAX_SEQUENCE_LENGTH,),
            dtype=tf.float32,
            name="Bug_TitleandDescription",
        )  # Bug Title and Description
        input_AST = Input(
            shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.float32, name="Bug_CodeSnippetAST"
        )  # Bug Code Snippet AST
        emb_Context = Embedding(
            input_dim=len(tk_context.word_index) + 2,
            input_length=MAX_SEQUENCE_LENGTH,
            output_dim=EMBEDDING_DIM,
        )(input_context)
        emb_AST = Embedding(
            input_dim=len(tk_AST.word_index) + 2,
            input_length=MAX_SEQUENCE_LENGTH,
            output_dim=EMBEDDING_DIM,
        )(input_AST)
        cat = concatenate([emb_Context, emb_AST])
        sd = SpatialDropout1D(0.5)(cat)
        # Bidirectional layer will enable our model to predict a missing word in a sequence,
        # So, using this feature will enable the model to look at the context on both the left and the right.
        bilstm = Bidirectional(LSTM(25, return_sequences=True))(sd)
        bn = BatchNormalization()(bilstm)
        drop = Dropout(0.5)(bn)
        maxpool = GlobalMaxPool1D()(drop)
        dense = Dense(50, activation="relu")(maxpool)
        final = Dense(noofbugtype, activation="sigmoid")(dense)
        Bil_LSTM_model = Model(inputs=[input_context, input_AST], outputs=[final])

        Bil_LSTM_model.compile(
            loss="binary_crossentropy",
            optimizer=Adam(Learningrate),
            metrics=[
                "accuracy",
                c_precision,
                c_recall,
                "AUC",
                c_fbeta,
                c_f1_macro,
                hammingloss,
            ],
        )

        csv_logger_bug = CSVLogger(filename + ".csv", append=True, separator=",")
        history = Bil_LSTM_model.fit(
            [x_train_context, x_train_AST],
            btype_y_train,
            callbacks=[tensorboard_callback, earlystop, csv_logger_bug],
            epochs=50,
            verbose=2,
            validation_split=0.1,
        )

        ## Evaluate RNN Single Model
        evaluate = Bil_LSTM_model.evaluate([x_test_context, x_test_AST], btype_y_test)
        df_evaluate = pd.DataFrame(
            columns=[
                "loss",
                "accuracy",
                "c_precision",
                "c_recall",
                "auc",
                "c_fbeta",
                "c_f1_macro",
                "hammingloss",
            ]
        )
        df_row = {
            "loss": evaluate[0],
            "accuracy": evaluate[1],
            "c_precision": evaluate[2],
            "c_recall": evaluate[3],
            "auc": evaluate[4],
            "c_fbeta": evaluate[5],
            "c_f1_macro": evaluate[6],
            "hammingloss": evaluate[7],
        }
        df_evaluate = df_evaluate.append(df_row, ignore_index=True)
        df_evaluate.to_csv(filename + "_eval.csv")

        filelog = open(filename + ".txt", "a")
        filelog.write("Endtime:" + str(datetime.now()))
        filelog.close()


In [ ]:
RNNModel(btype_labels_map,dev_labels_map,btype_y_train,dev_y_train,btype_y_test,dev_y_test,5,True)

Predict Developer
Epoch 1/50
27/27 - 3s - loss: 0.2907 - accuracy: 0.7737 - c_precision: 0.5607 - c_recall: 0.8425 - auc: 0.9021 - c_fbeta: 0.7145 - c_f1_macro: nan - hammingloss: 0.1100 - val_loss: 0.6307 - val_accuracy: 0.8387 - val_c_precision: 0.8220 - val_c_recall: 0.8373 - val_auc: 0.8961 - val_c_fbeta: 0.8343 - val_c_f1_macro: nan - val_hammingloss: 0.0215 - 3s/epoch - 129ms/step
Epoch 2/50
27/27 - 0s - loss: 0.0867 - accuracy: 0.8467 - c_precision: 0.8396 - c_recall: 0.8323 - auc: 0.9388 - c_fbeta: 0.8349 - c_f1_macro: nan - hammingloss: 0.0198 - val_loss: 0.6007 - val_accuracy: 0.8387 - val_c_precision: 0.8373 - val_c_recall: 0.8373 - val_auc: 0.9499 - val_c_fbeta: 0.8373 - val_c_f1_macro: nan - val_hammingloss: 0.0202 - 456ms/epoch - 17ms/step
Epoch 3/50
27/27 - 0s - loss: 0.0791 - accuracy: 0.8467 - c_precision: 0.8411 - c_recall: 0.8333 - auc: 0.9581 - c_fbeta: 0.8360 - c_f1_macro: nan - hammingloss: 0.0196 - val_loss: 0.5724 - val_accuracy: 0.8387 - val_c_precision: 0.8373

/tmp/ipykernel_2176299/945050786.py:159: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_evaluate = df_evaluate.append(df_row, ignore_index=True)
